In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize as wt 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import gensim
from gensim.models import Word2Vec

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# setting global variables to input datasets
datapathTrain = '/kaggle/input/commonlitreadabilityprize/train.csv'
datapathTest = '/kaggle/input/commonlitreadabilityprize/test.csv'

word2vecModel = gensim.models.KeyedVectors.load_word2vec_format("/kaggle/input/google-pretrain-model/GoogleNews-vectors-negative300.bin.gz", binary=True)

columns=['id','url','licence','text','score','stdev']
keepColumns = ['id','text','score','stdev']

In [ ]:
# function to read csv into dataframe
def csv2dataset(datapath,columns=columns,keepColumns=keepColumns):
    dataset = pd.read_csv(datapath,names=columns,skiprows=[0])
    return dataset[keepColumns]    

In [ ]:
# function to cleanup and tokenize the text 
def datasetTokenizer(dataset):
    dataset['tokenized_text'] = dataset['text']
    for i in range(dataset.shape[0]):
        sms = dataset['text'].iloc[i]
        # remove non alphabatic characters
        sms = re.sub('[^A-Za-z]', ' ', sms)
        # make words lowercase, because Go and go will be considered as two words
        sms = sms.lower()
        # tokenising
        tokenized_sms = wt(sms)
        # remove stop words and stemming
        sms_processed = []
        for word in tokenized_sms:
            if word not in set(stopwords.words('english')):
                sms_processed.append(stemmer.stem(word))

        sms_text = " ".join(sms_processed)
        dataset['tokenized_text'].iloc[i] = sms_text # update this line to avoid copy warning
    return dataset

In [ ]:
# creating the feature matrix 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def avg_feature_vector(sentence, model, num_features):
    words = sentence.replace('\n'," ").replace(',',' ').replace('.'," ").split()
    feature_vec = np.zeros((num_features,),dtype="float32")
    i=0
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError as error:
            feature_vec 
            i = i + 1
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words)- i)
    return feature_vec

def w2vsequencer(sentence, model, num_features, rating_mean, rating_stdev):
    words = sentence.replace('\n'," ").replace(',',' ').replace('.'," ").split()
    feature_vec = np.zeros((num_features,),dtype="float32")
    i=0
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError as error:
            feature_vec 
            i = i + 1
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words)- i)
    return feature_vec

def datasetVectorizer(dataset):
    vectorizer = CountVectorizer(max_features=1000) #Bag of words
    bow = vectorizer.fit_transform(dataset['tokenized_text']).toarray()
    dataset['BagOfWords_text'] = dataset['tokenized_text']

    vectorizer = TfidfVectorizer(max_features=1000) #TFIDF
    tfidf = vectorizer.fit_transform(dataset['tokenized_text']).toarray()
    dataset['TFIDF_text'] = dataset['tokenized_text']

    word2vec_train = np.zeros((dataset.shape[0],300),dtype="float32")
    dataset['Word2Vec_text'] = dataset['tokenized_text']

    for i in range(dataset.shape[0]):
        dataset['BagOfWords_text'].iloc[i] = bow[i]
        dataset['TFIDF_text'].iloc[i] = tfidf[i]
        dataset['Word2Vec_text'].iloc[i] = avg_feature_vector(dataset['tokenized_text'][i],word2vecModel, 300)

    return dataset



In [ ]:
datasetTrain = csv2dataset(datapathTrain)
datasetTrain = datasetTokenizer(datasetTrain)
datasetTrain = datasetVectorizer(datasetTrain)
datasetTrain.head()

In [ ]:
datasetTest = csv2dataset(datapathTest)
datasetTest = datasetTokenizer(datasetTest)
datasetTest = datasetVectorizer(datasetTest)
datasetTest.head()

In [ ]:
df = pd.Dataset(data)
df.to_csv(CLds, index = false)



